# Item relation matrices

### Get the data

In [65]:
import polars as pl

df_schema = {"session": pl.UInt32, "aid": pl.UInt32, "ts": pl.UInt32, "type": pl.Utf8}
df = pl.read_csv('../data/generated/train_events_last_half.csv', schema=df_schema)

print(df)

shape: (99_080_120, 4)
┌──────────┬─────────┬────────────┬────────┐
│ session  ┆ aid     ┆ ts         ┆ type   │
│ ---      ┆ ---     ┆ ---        ┆ ---    │
│ u32      ┆ u32     ┆ u32        ┆ str    │
╞══════════╪═════════╪════════════╪════════╡
│ 1        ┆ 723931  ┆ 1660544235 ┆ clicks │
│ 1        ┆ 1436439 ┆ 1660544269 ┆ clicks │
│ 1        ┆ 1693461 ┆ 1660544287 ┆ clicks │
│ 1        ┆ 1206554 ┆ 1660544299 ┆ clicks │
│ 2        ┆ 1110741 ┆ 1660546139 ┆ clicks │
│ …        ┆ …       ┆ …          ┆ …      │
│ 13925402 ┆ 1737908 ┆ 1661723987 ┆ clicks │
│ 13925403 ┆ 384045  ┆ 1661723976 ┆ clicks │
│ 13925403 ┆ 384045  ┆ 1661723986 ┆ clicks │
│ 13925404 ┆ 561560  ┆ 1661723983 ┆ clicks │
│ 13925404 ┆ 32070   ┆ 1661723994 ┆ clicks │
└──────────┴─────────┴────────────┴────────┘


### Click to click matrix
Click to click matrix is defined as the probabilities of other aids being clicked after the previous aid is clicked.
Click to click matrix is formed from the sub sessions since there is no point in counting subsequent clicks that are from a user coming back to site after a long time

We do multiple variations of the click to click matrix:

Next five clicks with time decay. Remove carts and orders from sessions.

In [7]:
subsequent_clicks_of_sessions = (
    df
    .filter(pl.col("type") == "clicks")
    .drop("type")
    .sort(["session", "ts"], descending=[False, False])
)

subsequent_clicks = pl.DataFrame({"session": [], "aid": [], "next_aid": [], "weight": []}, schema={"session": pl.UInt32, "aid": pl.UInt32, "next_aid": pl.UInt32, "weight": pl.Float32})

n_clicks_ahead = 5
for i in range(1, n_clicks_ahead+1):
    new_click_pairs = (
        # take the original clicks of sessions
        subsequent_clicks_of_sessions
        .with_columns(
            next_session=pl.col("session").shift(-i),
            next_aid=pl.col("aid").shift(-i),
            weight=pl.lit(1/i).cast(pl.Float32)
        )
        .filter(pl.col("session") == pl.col("next_session"))
        .drop(["ts", "next_session"])
    )
    subsequent_clicks = pl.concat([
        subsequent_clicks,
        new_click_pairs
    ])

# Sum all the weights for each click-click pair
click_to_click_count = (
    subsequent_clicks
    .group_by(["aid", "next_aid"])
    .agg(pl.col("weight").sum().alias("weighted_count"))
)

# Count total weight for each click
# Since we have weights in play we need to count weighed sum instead of count of rows
aid_clicks_total_count = (
    click_to_click_count
    .group_by("aid")
    .agg(pl.col("weighted_count").sum().alias("weighted_total_count"))
)


# Calculate the probabilities of items being clicked after another item has been clicked
click_to_click_matrix = (
    click_to_click_count
    .join(aid_clicks_total_count, on="aid", how="inner")
    .with_columns(
        probability = (pl.col("weighted_count") / pl.col("weighted_total_count")).cast(pl.Float32)
    )
    .drop(["weighted_count", "weighted_total_count"])
    .sort(["aid", "next_aid"])
    .rename({"aid": "click_aid", "next_aid": "next_click_aid"})
)

print(click_to_click_matrix)

shape: (111_717_938, 3)
┌───────────┬────────────────┬─────────────┐
│ click_aid ┆ next_click_aid ┆ probability │
│ ---       ┆ ---            ┆ ---         │
│ u32       ┆ u32            ┆ f32         │
╞═══════════╪════════════════╪═════════════╡
│ 0         ┆ 0              ┆ 0.021177    │
│ 0         ┆ 13759          ┆ 0.025413    │
│ 0         ┆ 31465          ┆ 0.006353    │
│ 0         ┆ 39551          ┆ 0.005083    │
│ 0         ┆ 53949          ┆ 0.012706    │
│ …         ┆ …              ┆ …           │
│ 1855602   ┆ 1693232        ┆ 0.050891    │
│ 1855602   ┆ 1762441        ┆ 0.010178    │
│ 1855602   ┆ 1768521        ┆ 0.010178    │
│ 1855602   ┆ 1783511        ┆ 0.076336    │
│ 1855602   ┆ 1855602        ┆ 0.063613    │
└───────────┴────────────────┴─────────────┘


In [6]:
# save to csv
click_to_click_matrix.write_csv("./click-to-click-matrix_only-clicks_5-subsequent-clicks-time-decay.csv")

Next five clicks with time decay. Remove carts and orders from sessions. Remove sessions with only 2 events.

In [16]:
subsequent_clicks_of_sessions = (
    df
    .filter(pl.col("type") == "clicks")
    .drop("type")
    .sort(["session", "ts"], descending=[False, False])
    .with_columns(session_event_count=pl.col("session").cum_count().over("session"))
    .with_columns(session_event_count=pl.col("session_event_count").max().over("session"))
    .filter(pl.col("session_event_count") > 3)
    # .filter(pl.col("session_event_count") <= 40)
    .drop("session_event_count")
)

subsequent_clicks = pl.DataFrame({"session": [], "aid": [], "next_aid": [], "weight": []}, schema={"session": pl.UInt32, "aid": pl.UInt32, "next_aid": pl.UInt32, "weight": pl.Float32})

n_clicks_ahead = 5
for i in range(1, n_clicks_ahead+1):
    new_click_pairs = (
        # take the original clicks of sessions
        subsequent_clicks_of_sessions
        .with_columns(
            next_session=pl.col("session").shift(-i),
            next_aid=pl.col("aid").shift(-i),
            weight=pl.lit(1/i).cast(pl.Float32)
        )
        .filter(pl.col("session") == pl.col("next_session"))
        .drop(["ts", "next_session"])
    )
    subsequent_clicks = pl.concat([
        subsequent_clicks,
        new_click_pairs
    ])

# Sum all the weights for each click-click pair
click_to_click_count = (
    subsequent_clicks
    .group_by(["aid", "next_aid"])
    .agg(pl.col("weight").sum().alias("weighted_count"))
)

# Count total weight for each click
# Since we have weights in play we need to count weighed sum instead of count of rows
aid_clicks_total_count = (
    click_to_click_count
    .group_by("aid")
    .agg(pl.col("weighted_count").sum().alias("weighted_total_count"))
)


# Calculate the probabilities of items being clicked after another item has been clicked
click_to_click_matrix = (
    click_to_click_count
    .join(aid_clicks_total_count, on="aid", how="inner")
    .with_columns(
        probability = (pl.col("weighted_count") / pl.col("weighted_total_count")).cast(pl.Float32)
    )
    .drop(["weighted_count", "weighted_total_count"])
    .sort(["aid", "next_aid"])
    .rename({"aid": "click_aid", "next_aid": "next_click_aid"})
)

print(click_to_click_matrix)

shape: (108_596_042, 3)
┌───────────┬────────────────┬─────────────┐
│ click_aid ┆ next_click_aid ┆ probability │
│ ---       ┆ ---            ┆ ---         │
│ u32       ┆ u32            ┆ f32         │
╞═══════════╪════════════════╪═════════════╡
│ 0         ┆ 0              ┆ 0.02173     │
│ 0         ┆ 13759          ┆ 0.026076    │
│ 0         ┆ 31465          ┆ 0.006519    │
│ 0         ┆ 39551          ┆ 0.005215    │
│ 0         ┆ 53949          ┆ 0.013038    │
│ …         ┆ …              ┆ …           │
│ 1855602   ┆ 1693232        ┆ 0.050891    │
│ 1855602   ┆ 1762441        ┆ 0.010178    │
│ 1855602   ┆ 1768521        ┆ 0.010178    │
│ 1855602   ┆ 1783511        ┆ 0.076336    │
│ 1855602   ┆ 1855602        ┆ 0.063613    │
└───────────┴────────────────┴─────────────┘


In [17]:
# save to csv
click_to_click_matrix.write_csv("./click-to-click-matrix_only-clicks_5-subsequent-clicks-time-decay_over-3-event-sessions.csv")

### Click to cart matrix
Click to cart matrix is defined as the probabilities of other aids being added to cart in the same sub session after an aid is clicked.

Time decay on carts happening later in the sessions

In [44]:
sessions = (
    df
    # Remove subsequent same type events on same aids
    .with_columns(
        next_session=pl.col("session").shift(-1),
        next_aid=pl.col("aid").shift(-1),
        next_type=pl.col("type").shift(-1),
    )
    .filter(pl.when((pl.col("session") == pl.col("next_session")) & (pl.col("type") == pl.col("next_type"))).then((pl.col("aid") != pl.col("next_aid"))).otherwise(True))
    .drop(["next_session", "next_aid", "next_type"])
    # Calculate the index of each event
    # Subsequent carts are considered to be in the same index
    .with_columns(cart_id=(pl.col("type") == "carts").rle_id())
    .with_columns(click_id=(pl.col("type") == "clicks").cum_sum())
    .with_columns(index=pl.struct("session", "cart_id", "click_id").rle_id())
    .drop(["cart_id", "click_id"])
)

# Get clicks and carts of sessions
clicks_sessions = (
    sessions
    .filter(pl.col("type") == "clicks")
    .rename({"ts": "click_ts", "aid": "click_aid", "index": "click_index"})
    .drop("type")
)

carts_of_sessions = (
    sessions
    .filter(pl.col("type") == "carts")
    .rename({"ts": "cart_ts", "aid": "cart_aid", "index": "cart_index"})
    .drop("type")
)

carts_after_clicks_in_sessions = (
    carts_of_sessions
    # Combine clicks and carts of sessions
    .join(clicks_sessions, on="session", how="inner")
    # Keep only carts that happened after clicks
    .filter(pl.col("click_ts") < pl.col("cart_ts"))
    # Weight the click-to-cart relation based in index. Next cart has weight 1/1, second 1/2, third 1/3, etc...
    .with_columns(weight=(1/(pl.col("cart_index")-pl.col("click_index"))).cast(pl.Float32))
    .drop(["click_ts", "cart_ts", "click_index", "cart_index"])
)

# Count how many same click to cart events there are
click_to_cart_count = (
    carts_after_clicks_in_sessions
    .group_by(["click_aid", "cart_aid"])
    .agg(pl.col("weight").sum().alias("weighted_count"))
    .sort("weighted_count", descending=True)
)

# Sum all the clicks for each aid
aid_clicks_total_count = (
    click_to_cart_count
    .group_by("click_aid")
    .agg(pl.sum("weighted_count").alias("weighted_total_count"))
    .sort("weighted_total_count", descending=True)
)

# Calculate the probabilities of items being added to cart after another item has been clicked
click_to_cart_matrix = (
    click_to_cart_count
    .join(aid_clicks_total_count, on="click_aid", how="inner")
    .with_columns(
        probability = pl.col("weighted_count") / pl.col("weighted_total_count")
    )
    .with_columns(pl.col("probability").cast(pl.Float32))
    .drop(["weighted_count", "weighted_total_count"])
    .sort(["click_aid", "cart_aid"])
)

print(click_to_cart_matrix)

# Check that probabilities sum to the amount of unique aids
print("Unique aids:", click_to_cart_matrix.select("click_aid").n_unique())
print("Total probability:", click_to_cart_matrix.select("probability").sum().select(pl.first()).item())

shape: (38_583_342, 3)
┌───────────┬──────────┬─────────────┐
│ click_aid ┆ cart_aid ┆ probability │
│ ---       ┆ ---      ┆ ---         │
│ u32       ┆ u32      ┆ f32         │
╞═══════════╪══════════╪═════════════╡
│ 0         ┆ 31465    ┆ 0.037618    │
│ 0         ┆ 45036    ┆ 0.008681    │
│ 0         ┆ 78027    ┆ 0.012539    │
│ 0         ┆ 197519   ┆ 0.022571    │
│ 0         ┆ 341875   ┆ 0.037618    │
│ …         ┆ …        ┆ …           │
│ 1855602   ┆ 1030452  ┆ 0.025716    │
│ 1855602   ┆ 1192996  ┆ 0.033064    │
│ 1855602   ┆ 1376245  ┆ 0.077149    │
│ 1855602   ┆ 1768521  ┆ 0.231447    │
│ 1855602   ┆ 1783511  ┆ 0.231447    │
└───────────┴──────────┴─────────────┘
Unique aids: 1390987
Total probability: 1390987.0


In [45]:
# save to csv
click_to_cart_matrix.write_csv("./click_to_cart_matrix_time-decay.csv")

### Click to order matrix
Click to order matrix is not done since it is not useful. Click to cart matrix works better than any click to order matrix.
Possible reasons: 1. Much more carts than orders. 2. Orders are usually done at the end of the session. Carts happen during the session so there is more clear relation between clicks and carts. than clicks and orders.

### Cart to cart matrix
Cart to cart matrix is defined as the probabilities of other aids being added to cart later in the same sub session where an aid is added to cart.

time decay

In [22]:
carts_of_sessions = (
    df
    .filter(pl.col("type") == "carts")
    .with_row_index()
    .rename({"ts": "cart_ts", "aid": "cart_aid", "index": "cart_index"})
)

next_carts_of_sessions = (
    carts_of_sessions
    .rename({"cart_ts": "next_cart_ts", "cart_aid": "next_cart_aid", "cart_index": "next_cart_index"})
)

# Find subsequent carts in the same session
subsequent_carts = (
    carts_of_sessions
    .join(next_carts_of_sessions, on="session", how="inner")
    .filter(pl.col("cart_ts") < pl.col("next_cart_ts"))
    .select(["session", "cart_aid", "next_cart_aid", "cart_index", "next_cart_index"])
    # Weight the cart-to-cart relation based in index. Next cart has weight 1/1, second 1/2, third 1/3, etc...
    .with_columns(weight=(1/(pl.col("next_cart_index")-pl.col("cart_index"))).cast(pl.Float32))
    .drop(["cart_index", "next_cart_index"])
)

# Sum all the weights for each cart-cart pair
cart_to_cart_count = (
    subsequent_carts
    .group_by(["cart_aid", "next_cart_aid"])
    .agg(pl.col("weight").sum().alias("weighted_count"))
)

# Count total weight for each cart
# Since we have weights in play we need to count weighed sum instead of count of rows
aid_carts_total_count = (
    subsequent_carts
    .group_by("cart_aid")
    .agg(pl.col("weight").sum().alias("weighted_total_count"))
)


# Calculate the weighted probabilities of items being added to cart after another item has been clicked
cart_to_cart_matrix = (
    cart_to_cart_count
    .join(aid_carts_total_count, on="cart_aid", how="inner")
    .with_columns(
        probability = (pl.col("weighted_count") / pl.col("weighted_total_count")).cast(pl.Float32)
    )
    .drop(["weighted_count", "weighted_total_count"])
    .sort(["cart_aid", "next_cart_aid"])
)

print(cart_to_cart_matrix)

# Check that probabilities sum to the amount of unique aids
print("Unique aids:", cart_to_cart_matrix.select("cart_aid").n_unique())
print("Total probability:", cart_to_cart_matrix.select("probability").sum().select(pl.first()).item())

shape: (11_933_532, 3)
┌──────────┬───────────────┬─────────────┐
│ cart_aid ┆ next_cart_aid ┆ probability │
│ ---      ┆ ---           ┆ ---         │
│ u32      ┆ u32           ┆ f32         │
╞══════════╪═══════════════╪═════════════╡
│ 3        ┆ 3             ┆ 0.334078    │
│ 3        ┆ 16778         ┆ 0.010841    │
│ 3        ┆ 67776         ┆ 0.010841    │
│ 3        ┆ 109499        ┆ 0.010841    │
│ 3        ┆ 117981        ┆ 0.010841    │
│ …        ┆ …             ┆ …           │
│ 1855601  ┆ 1158151       ┆ 0.047306    │
│ 1855601  ┆ 1326991       ┆ 0.036793    │
│ 1855601  ┆ 1566830       ┆ 0.11038     │
│ 1855601  ┆ 1700846       ┆ 0.066228    │
│ 1855601  ┆ 1786336       ┆ 0.05519     │
└──────────┴───────────────┴─────────────┘
Unique aids: 725333
Total probability: 725333.0


In [23]:
# save to csv
cart_to_cart_matrix.write_csv("./cart_to_cart_matrix_time-decay.csv")

### Cart to order matrix
Cart to order matrix is defined as the probabilities of other aids being ordered later in the same sub session where an aid is added to cart.

In [63]:
# orders and carts preceding orders are grouped together
grouped_orders_and_carts = (
    df
    .sort(["session", "ts"])
    .filter(pl.col("type") != "clicks")
    # Filter out sessions with no orders
    .group_by("session")
    .agg(pl.all(), pl.col("type").filter(pl.col("type") == "orders").n_unique().alias("has_orders"))
    .filter(pl.col("has_orders") == 1)
    .drop("has_orders")
    .explode(["aid", "ts", "type"])
    # Group orders and clicks
    .with_columns(order_timestamp_mask=pl.when(pl.col("type") == "orders").then(pl.col("ts")).otherwise(0).cast(pl.UInt32))
    .with_columns(group=pl.struct("session", "order_timestamp_mask").rle_id().cast(pl.UInt32))
    .with_columns(group=pl.when(pl.col("type") == "carts").then(pl.col("group") + 1).otherwise(pl.col("group")))
    .drop(["order_timestamp_mask", "session", "ts"])
    # Remove duplicate clicks and orders of groups
    .unique(maintain_order=True)
)

cart_to_order_count = (
    grouped_orders_and_carts
    .filter(pl.col("type") == "carts")
    .drop("type")
    .rename({"aid": "cart_aid"})
    .join(
        (
            grouped_orders_and_carts
            .filter(pl.col("type") == "orders")
            .drop("type")
            .rename({"aid": "order_aid"})
        ), on="group", how="inner")
    .group_by(["cart_aid", "order_aid"])
    .agg(pl.len().alias("count"))
)

# Sum all the carts for each aid
aid_carts_total_count = (
    cart_to_order_count
    .group_by("cart_aid")
    .agg(pl.sum("count").alias("total_count"))
    .sort("total_count", descending=True)
)

# Calculate the probabilities of items being ordered after another item has been added to cart
cart_to_order_matrix = (
    cart_to_order_count
    .join(aid_carts_total_count, on="cart_aid")
    .with_columns(
        probability = pl.col("count") / pl.col("total_count")
    )
    .with_columns(pl.col("probability").cast(pl.Float32))
    .drop(["count", "total_count"])
    .sort(["cart_aid", "order_aid"])
)

print(cart_to_order_matrix)

# Check that probabilities sum to the amount of unique aids
print("Unique aids:", cart_to_order_matrix.select("cart_aid").n_unique())
print("Total probability:", cart_to_order_matrix.select("probability").sum().select(pl.first()).item())

shape: (6_051_171, 3)
┌──────────┬───────────┬─────────────┐
│ cart_aid ┆ order_aid ┆ probability │
│ ---      ┆ ---       ┆ ---         │
│ u32      ┆ u32       ┆ f32         │
╞══════════╪═══════════╪═════════════╡
│ 3        ┆ 3         ┆ 0.538462    │
│ 3        ┆ 22107     ┆ 0.019231    │
│ 3        ┆ 67776     ┆ 0.019231    │
│ 3        ┆ 138431    ┆ 0.019231    │
│ 3        ┆ 164205    ┆ 0.019231    │
│ …        ┆ …         ┆ …           │
│ 1855601  ┆ 1360116   ┆ 0.1         │
│ 1855601  ┆ 1486834   ┆ 0.1         │
│ 1855601  ┆ 1566830   ┆ 0.1         │
│ 1855601  ┆ 1700846   ┆ 0.1         │
│ 1855601  ┆ 1786336   ┆ 0.1         │
└──────────┴───────────┴─────────────┘
Unique aids: 521691
Total probability: 521690.96875


In [64]:
# save to csv
cart_to_order_matrix.write_csv("./cart_to_order_matrix.csv")

Cart to order with time decay

In [ ]:
sub_sessions = (
    get_sub_sessions(with_next_event=False)
    # Filter out clicks
    .filter(pl.col("type") != "clicks")
    .with_columns(index=(pl.col("sub_session") + (pl.col("type") == "orders").cum_sum()))
)

# Get clicks and orders of sessions
carts_of_sub_sessions = (
    sub_sessions
    .filter(pl.col("type") == "carts")
    .rename({"ts": "cart_ts", "aid": "cart_aid", "index": "cart_index"})
)

orders_of_sub_sessions = (
    sub_sessions
    .filter(pl.col("type") == "orders")
    .rename({"ts": "order_ts", "aid": "order_aid", "index": "order_index"})
)

orders_after_carts_in_sub_sessions = (
    orders_of_sub_sessions
    # Combine carts and orders of sessions
    .join(carts_of_sub_sessions, on="sub_session", how="inner")
    # Keep only orders that happened after carts
    .filter(pl.col("cart_ts") < pl.col("order_ts"))
    .select(["sub_session", "cart_aid", "order_aid", "cart_index", "order_index"])
    # Weight the cart-to-order relation based in index. Next order has weight 1/1, second 1/2, third 1/3, etc...
    .with_columns(weight=(1/(pl.col("order_index")-pl.col("cart_index"))).cast(pl.Float32))
    .drop(["cart_index", "order_index"])
)

# Sum all the weights for each cart-order pair
cart_to_order_count = (
    orders_after_carts_in_sub_sessions
    .group_by(["cart_aid", "order_aid"])
    .agg(pl.col("weight").sum().alias("weighted_count"))
)

# Count total weight for each cart
# Since we have weights in play we need to count weighed sum instead of count of rows
aid_carts_total_count = (
    orders_after_carts_in_sub_sessions
    .group_by("cart_aid")
    .agg(pl.col("weight").sum().alias("weighted_total_count"))
)


# Calculate the weighted probabilities of items being added to order after another item has been clicked
cart_to_order_matrix = (
    cart_to_order_count
    .join(aid_carts_total_count, on="cart_aid", how="inner")
    .with_columns(
        probability = (pl.col("weighted_count") / pl.col("weighted_total_count")).cast(pl.Float32)
    )
    .drop(["weighted_count", "weighted_total_count"])
    .sort(["cart_aid", "order_aid"])
)

print(cart_to_order_matrix)

# Check that probabilities sum to the amount of unique aids
print("Unique aids:", cart_to_order_matrix.select("cart_aid").n_unique())
print("Total probability:", cart_to_order_matrix.select("probability").sum().select(pl.first()).item())

In [ ]:
# save to csv
cart_to_order_matrix.write_csv("./cart_to_order_matrix_time_decay.csv")

Cart to order from whole sessions

In [42]:
carts_and_orders_df = (
    df
    .explode("events")
    .unnest("events")
    .filter(pl.col("type") != "clicks")
    .sort(["session", "ts"], descending=[False, False])
    .drop("ts")
    # Make groups based on types and then make sure that session boundaries are not crossed
    .with_columns(group=(pl.col("type") == "orders").rle_id().cast(pl.UInt32))
    .with_columns(group=pl.struct("session", "group").rle_id())
    # Remove multiple carts and orders in the same group
    .unique()
    .lazy()
)

cart_to_order = (
     carts_and_orders_df
    .join(carts_and_orders_df.rename({"group": "next_group", "aid": "next_aid", "type": "next_type" }), on="session", how="inner")
    .filter(pl.col("group") < pl.col("next_group"))
    .filter((pl.col("type") == "carts") & (pl.col("next_type") == "orders"))
    .drop(["type", "next_type"])
    # time decay
    .with_columns(weight = (1 / (pl.col("next_group") - pl.col("group"))).cast(pl.Float32))
    .drop(["group", "next_group"])
)

cart_to_order_count = (
    cart_to_order
    .group_by(["aid", "next_aid"])
    # time decay
    .agg(pl.col("weight").sum().alias("weighted_count"))
    # no time decay
    # .agg(pl.len().alias("count"))
    # .sort("count", descending=True)
)

cart_aid_total_count = (
    cart_to_order_count
    .group_by("aid")
    # time decay
    .agg(pl.col("weighted_count").sum().alias("weighted_total_count"))
    # no time decay
    # .agg(pl.sum("count").alias("total_count"))
    # .sort("total_count", descending=True)
)

cart_to_order_matrix_df = (
    cart_to_order_count
    .join(cart_aid_total_count, on="aid")
    .with_columns(
        probability = pl.col("weighted_count") / pl.col("weighted_total_count")
    )
    .with_columns(pl.col("probability").cast(pl.Float32))
    .drop(["weighted_count", "weighted_total_count"])
    .sort(["aid", "next_aid"])
    .collect(streaming=True)
)

print(cart_to_order_matrix_df)

# save to csv
cart_to_order_matrix_df.write_csv("./cart_to_order_matrix_whole_sessions_time_decay.csv")

shape: (28_135_990, 3)
┌─────────┬──────────┬─────────────┐
│ aid     ┆ next_aid ┆ probability │
│ ---     ┆ ---      ┆ ---         │
│ u32     ┆ u32      ┆ f32         │
╞═════════╪══════════╪═════════════╡
│ 3       ┆ 3        ┆ 0.366832    │
│ 3       ┆ 22107    ┆ 0.00957     │
│ 3       ┆ 61428    ┆ 0.00957     │
│ 3       ┆ 67776    ┆ 0.00957     │
│ 3       ┆ 68426    ┆ 0.00319     │
│ …       ┆ …        ┆ …           │
│ 1855601 ┆ 1486834  ┆ 0.062241    │
│ 1855601 ┆ 1566830  ┆ 0.062241    │
│ 1855601 ┆ 1700846  ┆ 0.082988    │
│ 1855601 ┆ 1712873  ┆ 0.062241    │
│ 1855601 ┆ 1786336  ┆ 0.062241    │
└─────────┴──────────┴─────────────┘


### Order to click matrix
Order to click matrix is defined as the probabilities of other aids being clicked immediately after an aid is ordered.

In [ ]:
sub_sessions = get_sub_sessions(with_next_event=False)

# Get orders and clicks of sessions
orders_of_sub_sessions = get_orders_of_sub_session(sub_sessions)

clicks_of_sub_sessions = get_clicks_of_sub_session(sub_sessions)

clicks_after_orders_in_sub_sessions = (
    clicks_of_sub_sessions
    # Combine orders and clicks of sessions
    .join(orders_of_sub_sessions, on="sub_session", how="inner")
    # Keep only clicks that happened after orders
    .filter(pl.col("order_ts") < pl.col("click_ts"))
    .select(["sub_session", "order_aid", "click_aid"])
)

# Count how many same order to click events there are
order_to_click_count = (
    clicks_after_orders_in_sub_sessions
    .group_by(["order_aid", "click_aid"])
    .agg(pl.len().alias("count"))
    .sort("count", descending=True)
)

# Sum all the orders for each aid
aid_orders_total_count = (
    order_to_click_count
    .group_by("order_aid")
    .agg(pl.sum("count").alias("total_count"))
    .sort("total_count", descending=True)
)

# Calculate the probabilities of items being clicked immediately after another item has been ordered
order_to_click_matrix = (
    order_to_click_count
    .join(aid_orders_total_count, on="order_aid")
    .with_columns(
        probability = pl.col("count") / pl.col("total_count")
    )
    .with_columns(pl.col("probability").cast(pl.Float32))
    .drop(["count", "total_count"])
    .sort(["order_aid", "click_aid"])
)

print(order_to_click_matrix)

# Check that probabilities sum to the amount of unique aids
print("Unique aids:", order_to_click_matrix.select("order_aid").n_unique())
print("Total probability:", order_to_click_matrix.select("probability").sum().select(pl.first()).item())

In [ ]:
# save to csv
order_to_click_matrix.write_csv("./order_to_click_matrix.csv")

### Order to cart matrix
Order to cart matrix is defined as the probabilities of other aids being added to cart later in the same sub session where an aid is ordered.s

In [ ]:
sub_sessions = get_sub_sessions(with_next_event=False)

# Get orders and carts of sessions
orders_of_sub_sessions = get_orders_of_sub_session(sub_sessions)

carts_of_sub_sessions = get_carts_of_sub_session(sub_sessions)

carts_after_orders_in_sub_sessions = (
    carts_of_sub_sessions
    # Combine orders and carts of sessions
    .join(orders_of_sub_sessions, on="sub_session", how="inner")
    # Keep only carts that happened after orders
    .filter(pl.col("order_ts") < pl.col("cart_ts"))
    .select(["sub_session", "order_aid", "cart_aid"])
)

# Count how many same order to cart events there are
order_to_cart_count = (
    carts_after_orders_in_sub_sessions
    .group_by(["order_aid", "cart_aid"])
    .agg(pl.len().alias("count"))
    .sort("count", descending=True)
)

# Sum all the orders for each aid
aid_orders_total_count = (
    order_to_cart_count
    .group_by("order_aid")
    .agg(pl.sum("count").alias("total_count"))
    .sort("total_count", descending=True)
)

# Calculate the probabilities of items being added to cart after another item has been ordered
order_to_cart_matrix = (
    order_to_cart_count
    .join(aid_orders_total_count, on="order_aid")
    .with_columns(
        probability = pl.col("count") / pl.col("total_count")
    )
    .with_columns(pl.col("probability").cast(pl.Float32))
    .drop(["count", "total_count"])
    .sort(["order_aid", "cart_aid"])
)

print(order_to_cart_matrix)

# Check that probabilities sum to the amount of unique aids
print("Unique aids:", order_to_cart_matrix.select("order_aid").n_unique())
print("Total probability:", order_to_cart_matrix.select("probability").sum().select(pl.first()).item())

In [ ]:
# save to csv
order_to_cart_matrix.write_csv("./order_to_cart_matrix.csv")

Order to cart with time decay

In [ ]:
sub_sessions = (
    get_sub_sessions(with_next_event=False)
    # Filter out clicks
    .filter(pl.col("type") != "clicks")
    .with_columns(index=(pl.col("sub_session") + (pl.col("type") == "cart").cum_sum()))
)

# Get clicks and orders of sessions
orders_of_sub_sessions = (
    sub_sessions
    .filter(pl.col("type") == "orders")
    .rename({"ts": "order_ts", "aid": "order_aid", "index": "order_index"})
)

carts_of_sub_sessions = (
    sub_sessions
    .filter(pl.col("type") == "carts")
    .rename({"ts": "cart_ts", "aid": "cart_aid", "index": "cart_index"})
)

carts_after_orders_in_sub_sessions = (
    carts_of_sub_sessions
    # Combine carts and orders of sessions
    .join(orders_of_sub_sessions, on="sub_session", how="inner")
    # Keep only orders that happened after carts
    .filter(pl.col("order_ts") < pl.col("cart_ts"))
    .select(["sub_session", "cart_aid", "order_aid", "cart_index", "order_index"])
    # Weight the cart-to-order relation based in index. Next order has weight 1/1, second 1/2, third 1/3, etc...
    .with_columns(weight=(1/(pl.col("cart_index")-pl.col("order_index"))).cast(pl.Float32))
    .drop(["cart_index", "order_index"])
)

# Sum all the weights for each order-cart pair
order_to_cart_count = (
    orders_after_carts_in_sub_sessions
    .group_by(["order_aid", "cart_aid"])
    .agg(pl.col("weight").sum().alias("weighted_count"))
)

# Count total weight for each cart
# Since we have weights in play we need to count weighed sum instead of count of rows
aid_orders_total_count = (
    orders_after_carts_in_sub_sessions
    .group_by("order_aid")
    .agg(pl.col("weight").sum().alias("weighted_total_count"))
)


# Calculate the weighted probabilities of items being added to cart after another item has been ordered
order_to_cart_matrix = (
    order_to_cart_count
    .join(aid_orders_total_count, on="order_aid", how="inner")
    .with_columns(
        probability = (pl.col("weighted_count") / pl.col("weighted_total_count")).cast(pl.Float32)
    )
    .drop(["weighted_count", "weighted_total_count"])
    .sort(["order_aid", "cart_aid"])
)

print(order_to_cart_matrix)

# Check that probabilities sum to the amount of unique aids
print("Unique aids:", order_to_cart_matrix.select("order_aid").n_unique())
print("Total probability:", order_to_cart_matrix.select("probability").sum().select(pl.first()).item())

In [ ]:
# save to csv
order_to_cart_matrix.write_csv("./order_to_cart_matrix_time_decay.csv")

### Order to order matrix
Order to order matrix is defined as the probabilities of other aids being ordered later in the same sub session where an aid is ordered.s

In [ ]:
sub_sessions = get_sub_sessions(with_next_event=False)

# Get orders of sessions
orders_of_sub_sessions = get_orders_of_sub_session(sub_sessions)

next_orders_of_sub_sessions = (
    orders_of_sub_sessions
    .rename({"order_ts": "next_order_ts", "order_aid": "next_order_aid"})
)

# Find subsequent orders in the same session
subsequent_orders = (
    orders_of_sub_sessions
    .join(next_orders_of_sub_sessions, on="sub_session", how="inner")
    .filter(pl.col("order_ts") < pl.col("next_order_ts"))
    .select(["sub_session", "order_aid", "next_order_aid"])
)

# Count how many same order to order events there are
subsequent_orders_count = (
    subsequent_orders
    .group_by(["order_aid", "next_order_aid"])
    .agg(pl.len().alias("count"))
    .sort("count", descending=True)
)

# Sum all the orders for each aid
aid_orders_total_count = (
    subsequent_orders_count
    .group_by("order_aid")
    .agg(pl.sum("count").alias("total_count"))
    .sort("total_count", descending=True)
)

# Calculate the probabilities of items being ordered after another item has been ordered
order_to_order_matrix = (
    subsequent_orders_count
    .join(aid_orders_total_count, on="order_aid")
    .with_columns(
        probability = pl.col("count") / pl.col("total_count")
    )
    .with_columns(pl.col("probability").cast(pl.Float32))
    .drop(["count", "total_count"])
    .sort(["order_aid", "next_order_aid"])
)

print(order_to_order_matrix)

# Check that probabilities sum to the amount of unique aids
print("Unique aids:", order_to_order_matrix.select("order_aid").n_unique())
print("Total probability:", order_to_order_matrix.select("probability").sum().select(pl.first()).item())

In [ ]:
# save to csv
order_to_order_matrix.write_csv("./order_to_order_matrix.csv")

Order to order with time decay

In [ ]:
orders_of_sub_sessions = (
    get_sub_sessions(with_next_event=False)
    .filter(pl.col("type") == "orders")
    .with_row_index()
    .rename({"ts": "order_ts", "aid": "order_aid", "index": "order_index"})
)

next_orders_of_sub_sessions = (
    orders_of_sub_sessions
    .rename({"order_ts": "next_order_ts", "order_aid": "next_order_aid", "order_index": "next_order_index"})
)

# Find subsequent orders in the same session
subsequent_orders = (
    orders_of_sub_sessions
    .join(next_orders_of_sub_sessions, on="sub_session", how="inner")
    .filter(pl.col("order_ts") < pl.col("next_order_ts"))
    .select(["sub_session", "order_aid", "next_order_aid", "order_index", "next_order_index"])
    # Weight the order-to-order relation based in index. Next order has weight 1/1, second 1/2, third 1/3, etc...
    .with_columns(weight=(1/(pl.col("next_order_index")-pl.col("order_index"))).cast(pl.Float32))
    .drop(["order_index", "next_order_index"])
)

# Sum all the weights for each order-order pair
order_to_order_count = (
    subsequent_orders
    .group_by(["order_aid", "next_order_aid"])
    .agg(pl.col("weight").sum().alias("weighted_count"))
)

# Count total weight for each order
# Since we have weights in play we need to count weighed sum instead of count of rows
aid_orders_total_count = (
    subsequent_orders
    .group_by("order_aid")
    .agg(pl.col("weight").sum().alias("weighted_total_count"))
)


# Calculate the weighted probabilities of items being ordered after another item has been ordered
order_to_order_matrix = (
    order_to_order_count
    .join(aid_orders_total_count, on="order_aid", how="inner")
    .with_columns(
        probability = (pl.col("weighted_count") / pl.col("weighted_total_count")).cast(pl.Float32)
    )
    .drop(["weighted_count", "weighted_total_count"])
    .sort(["order_aid", "next_order_aid"])
)

print(order_to_order_matrix)

# Check that probabilities sum to the amount of unique aids
print("Unique aids:", order_to_order_matrix.select("order_aid").n_unique())
print("Total probability:", order_to_order_matrix.select("probability").sum().select(pl.first()).item())

In [ ]:
# save to csv
order_to_order_matrix.write_csv("./order_to_order_matrix_time_decay.csv")

Order to order whole sessions

In [46]:
orders_df = (
    df
    .explode("events")
    .unnest("events")
    .sort(["session", "ts"], descending=[False, False])
    .drop("ts")
    # Make groups based on types and then make sure that session boundaries are not crossed
    .with_columns(group=(pl.col("type") == "orders").rle_id().cast(pl.UInt32))
    .with_columns(group=pl.struct("session", "group").rle_id())
    .filter(pl.col("type") == "orders")
    .drop("type")
    # Remove multiple orders in the same group
    .unique()
    .lazy()
)

order_to_order = (
    orders_df
    .join(orders_df.rename({"group": "next_group", "aid": "next_aid"}), on="session", how="inner")
    .filter(pl.col("group") <= pl.col("next_group"))
    # Filter out same items ordered multiple times at the same time
    .filter((pl.col("group") != pl.col("next_group")) | (pl.col("aid") != pl.col("next_aid")))
    # time decay
    .with_columns(weight = (1 / (pl.col("next_group") - pl.col("group") + 1)).cast(pl.Float32))
    .drop(["group", "next_group"])
)

# Sum all the weights for each order-order pair
order_to_order_count = (
    order_to_order
    .group_by(["aid", "next_aid"])
    .agg(pl.col("weight").sum().alias("weighted_count"))
)

# Count total weight for each order
# Since we have weights in play we need to count weighed sum instead of count of rows
aid_orders_total_count = (
    order_to_order
    .group_by("aid")
    .agg(pl.col("weight").sum().alias("weighted_total_count"))
)


# Calculate the weighted probabilities of items being ordered after another item has been ordered
order_to_order_matrix = (
    order_to_order_count
    .join(aid_orders_total_count, on="aid", how="inner")
    .with_columns(
        probability = (pl.col("weighted_count") / pl.col("weighted_total_count")).cast(pl.Float32)
    )
    .drop(["weighted_count", "weighted_total_count"])
    .sort(["aid", "next_aid"])
    .rename({"aid": "order_aid", "next_aid": "next_order_aid"})
    .sort(["probability", "order_aid", "next_order_aid"], descending=True)
    .collect(streaming=True)
)

print(order_to_order_matrix)
order_to_order_matrix.write_csv("./order_to_order_matrix_whole_sessions_time_decay.csv")


# # Check that probabilities sum to the amount of unique aids
# print("Unique aids:", order_to_order_matrix.select("order_aid").n_unique())
# print("Total probability:", order_to_order_matrix.select("probability").sum().select(pl.first()).item())

shape: (18_196_971, 3)
┌───────────┬────────────────┬─────────────┐
│ order_aid ┆ next_order_aid ┆ probability │
│ ---       ┆ ---            ┆ ---         │
│ u32       ┆ u32            ┆ f32         │
╞═══════════╪════════════════╪═════════════╡
│ 1836735   ┆ 1836735        ┆ 1.0         │
│ 1835249   ┆ 1740171        ┆ 1.0         │
│ 1798478   ┆ 1798478        ┆ 1.0         │
│ 1718110   ┆ 1718110        ┆ 1.0         │
│ 1673053   ┆ 37525          ┆ 1.0         │
│ …         ┆ …              ┆ …           │
│ 231487    ┆ 644882         ┆ 0.000001    │
│ 846545    ┆ 1094510        ┆ 0.000001    │
│ 756588    ┆ 1711735        ┆ 0.000001    │
│ 846545    ┆ 444504         ┆ 0.000001    │
│ 846545    ┆ 1252537        ┆ 0.000001    │
└───────────┴────────────────┴─────────────┘


### Order incompatibility matrix
Order incompatibility matrix is defined as links between items which are often ordered with same items but are never ordered together.

In [66]:
grouped_orders = (
    df
    .filter(pl.col("type") == "orders")
    # Group orders by session and timestamp
    .with_columns(order_id=pl.struct("session", "ts").rle_id().cast(pl.UInt32))
    .drop("type")
    # Remove duplicates from same order
    .unique(subset=["order_id", "aid"], maintain_order=True)
    # Count in how many orders each aid appears
    .group_by("aid", maintain_order=True)
    .agg(pl.exclude("aid"), pl.len().alias("aid_order_count"))
    # Filter out aids with very few orders
    .filter(pl.col("aid_order_count") > 1)
    .explode(["order_id"])
    .sort(["order_id"])
)

print(grouped_orders)
print(grouped_orders.select("aid_order_count").describe())

order_pairs = (
    grouped_orders
    .drop(["session", "ts"])
    .rename({"aid": "aid_1", "aid_order_count": "aid_1_order_count"})
    .join(grouped_orders.drop(["session", "ts"]).rename({"aid": "aid_2", "aid_order_count": "aid_2_order_count"}), on="order_id", how="inner")
    .filter(pl.col("aid_1") != pl.col("aid_2"))
    .group_by(["aid_1", "aid_2"])
    .agg(pl.all(), pl.len().alias("aid_pair_order_count"))
    # Filter out aid pairs with very few orders
    .filter(pl.col("aid_pair_order_count") > 1)
    .sort("aid_pair_order_count", descending=True)
)

print(order_pairs)
print(order_pairs.select("aid_pair_order_count").describe())

shape: (2_041_966, 5)
┌─────────┬─────────────────────────────┬─────────────────────────────┬──────────┬─────────────────┐
│ aid     ┆ session                     ┆ ts                          ┆ order_id ┆ aid_order_count │
│ ---     ┆ ---                         ┆ ---                         ┆ ---      ┆ ---             │
│ u32     ┆ list[u32]                   ┆ list[u32]                   ┆ u32      ┆ u32             │
╞═════════╪═════════════════════════════╪═════════════════════════════╪══════════╪═════════════════╡
│ 543308  ┆ [20, 611179, … 13662569]    ┆ [1661552940, 1660733747, …  ┆ 0        ┆ 55              │
│         ┆                             ┆ 166…                        ┆          ┆                 │
│ 1670763 ┆ [45, 99348, … 13713227]     ┆ [1660567026, 1661601399, …  ┆ 1        ┆ 54              │
│         ┆                             ┆ 166…                        ┆          ┆                 │
│ 772148  ┆ [47, 30771, … 13862652]     ┆ [1661024972, 1661620424, … 

In [82]:
# orders and clicks preceding orders are grouped together
grouped_orders_and_clicks = (
    df
    .sort(["session", "ts"])
    .filter(pl.col("type") != "carts")
    # Filter out sessions with no orders
    .group_by("session")
    .agg(pl.all(), pl.col("type").filter(pl.col("type") == "orders").n_unique().alias("has_orders"))
    .filter(pl.col("has_orders") == 1)
    .drop("has_orders")
    .explode(["aid", "ts", "type"])
    # Group orders and clicks
    .with_columns(order_timestamp_mask=pl.when(pl.col("type") == "orders").then(pl.col("ts")).otherwise(0).cast(pl.UInt32))
    .with_columns(group=pl.struct("session", "order_timestamp_mask").rle_id().cast(pl.UInt32))
    .with_columns(group=pl.when(pl.col("type") == "clicks").then(pl.col("group") + 1).otherwise(pl.col("group")))
    .drop(["order_timestamp_mask", "session", "ts"])
    # Remove duplicate clicks and orders of groups
    .unique(maintain_order=True)
)

# Used for checking that items which share context have not been ordered together
aids_ordered_together = (
    grouped_orders_and_clicks
    .filter(pl.col("type") == "orders")
    .drop("type")
    .rename({"aid": "aid_1"})
    .join(grouped_orders_and_clicks.filter(pl.col("type") == "orders").drop("type").rename({"aid": "aid_2"}), on="group", how="inner")
    .filter(pl.col("aid_1") != pl.col("aid_2"))
    .group_by(["aid_1", "aid_2"])
    .agg(pl.len().alias("count"))
    # TODO: Play around with this value. How many can be considered as exception?
    .filter(pl.col("count") > 2)
    .sort("count", descending=True)
)
# print(aids_ordered_together)

# Used for determining which aids have enough orders to be considered for incompatibility
top_n_ordered_aids = 0.02
aids_with_enough_orders = (
    grouped_orders_and_clicks
    .filter(pl.col("type") == "orders")
    .drop(["type", "group"])
    # Count in how many orders each aid appears
    .group_by("aid", maintain_order=True)
    .agg(pl.len().alias("aid_order_count"))
    .sort("aid_order_count", descending=True)
    .filter(pl.col("aid_order_count") > pl.col("aid_order_count").quantile((1-top_n_ordered_aids), interpolation="midpoint"))
    .select("aid")
)


# Remove aids that have too few orders
# Both clicks and orders are removed to make sure that there is as little noise as possible
grouped_orders_and_clicks = (
    grouped_orders_and_clicks
    .join(aids_with_enough_orders, on="aid", how="semi")
)


# Used for determining the similarity of items based on their context
order_contexts = (
    grouped_orders_and_clicks
    .filter(pl.col("type") == "clicks")
    .drop("type")
    .rename({"aid": "click_aid"})
    .join(grouped_orders_and_clicks.filter(pl.col("type") == "orders").drop("type").rename({"aid": "order_aid"}), on="group", how="inner")
    .drop("group")
    .group_by(["order_aid", "click_aid"])
    .agg(pl.len().alias("count"))
    # Filter out pairs that appear only once meaning they are part of only one session
    .filter(pl.col("count") > 1)
    # Create contexts
    .group_by("order_aid")
    .agg(pl.col("click_aid").unique().sort().alias("context"))
)
print(order_contexts)

order_contexts_dict = (
    order_contexts
    .with_columns(order_aid=pl.col("order_aid").cast(str))
    .transpose(column_names="order_aid")
    .to_dict(as_series=False)
)

# Find all incompatible pairs by finding aids that have never been ordered together but have common aids that are often clicked before ordering
candidate_incompatible_pairs = []

order_context_rows = order_contexts.rows()
len_order_context = len(order_context_rows)
print("Start")
for index, row in enumerate(order_context_rows):
    if (index+1) % 10000 == 0:
        print(f"row {index+1} / {len_order_context}")

    order_aid = row[0]
    order_aid_context = row[1]

    for click_aid in order_aid_context:
        click_aid_context = order_contexts_dict.get(str(click_aid))
        if click_aid_context is not None:
            click_aid_context = click_aid_context[0]
            # Jaccard similarity
            similarity = len(set(order_aid_context).intersection(click_aid_context)) / len(set(order_aid_context).union(click_aid_context))
            if similarity > 0.8:
                candidate_incompatible_pairs.append((order_aid, click_aid))

print("Done")
print()

incompatible_df = (
    pl.DataFrame(
        data=candidate_incompatible_pairs,
        orient='row',
        schema={"aid": pl.UInt32, "incompatible_aid": pl.UInt32}
    )
    .filter(pl.col("aid") != pl.col("incompatible_aid"))
    .unique()
)
print(incompatible_df)

incompatible_df = (
    incompatible_df
    # Filter out pairs that are too often bought together
    .join(aids_ordered_together, left_on=["aid", "incompatible_aid"], right_on=["aid_1", "aid_2"], how="anti")
    .sort(["aid", "incompatible_aid"])
)
print(incompatible_df)

shape: (8_850, 2)
┌───────────┬─────────────────────────────┐
│ order_aid ┆ context                     │
│ ---       ┆ ---                         │
│ u32       ┆ list[u32]                   │
╞═══════════╪═════════════════════════════╡
│ 1328763   ┆ [70160, 169149, … 1747792]  │
│ 985351    ┆ [53600, 83604, … 1839274]   │
│ 174368    ┆ [174368, 258074, … 1836036] │
│ 1095186   ┆ [215620, 284139, … 1796802] │
│ 342019    ┆ [6851, 18185, … 1805466]    │
│ …         ┆ …                           │
│ 258074    ┆ [174368, 258074, … 1836036] │
│ 954655    ┆ [26525, 76799, … 1683038]   │
│ 557988    ┆ [557988, 1487177]           │
│ 909668    ┆ [314514, 411195, … 1840890] │
│ 1589314   ┆ [718019, 807430, … 1833626] │
└───────────┴─────────────────────────────┘
Start
Done

shape: (219, 2)
┌─────────┬──────────────────┐
│ aid     ┆ incompatible_aid │
│ ---     ┆ ---              │
│ u32     ┆ u32              │
╞═════════╪══════════════════╡
│ 1619389 ┆ 1336095          │
│ 1731425 ┆ 901329  

In [83]:
# Write to csv
incompatible_df.write_csv("./incompatible_matrix_7.csv")